<a href="https://colab.research.google.com/github/jsoldadomagraner/TReND-CaMinA/blob/main/notebooks/Rwanda24/18-Thu-ProbModels/probabilistic_models_LDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Probabilistic models: Linear Dynamical Systems**

<img align="left" width="100" src="https://raw.githubusercontent.com/trendinafrica/TReND-CaMinA/main/images/CaMinA_logo.png">

### **TReND-CaMinA: Computational Neuroscience and Machine Learning basics school, Rwanda 2024**
#### **Content creator:** Lea Duncker, Stanford University
#### **Adapted by:** Joana Soldado Magraner, Carnegie Mellon University

---

###**Learning objectives:**
* Generate time series data using an LDS, a simple probabilistic linear model
* Gain intuitions about how complex high-dimensional neural data, and in particular neural trajectories, can be described with simple low-dimensional dynamical models
* Apply algorithms to fit LDS models to time series data recorded from the brain

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm
import scipy

## **Exercise 1:** Exploring properties of linear dynamics

First, we will try to get some intuition about how linear dynamics and their properties shape the evolution of states over time. To do this, we will construct two-dimensional linear dynamics with different parameter settings, look at their eigenvalues and eigenvectors and simulate how the system evolves forward in time.

A linear dynamical system evolves in time according to the equation
$$ \boldsymbol{x}_{t+1} = A \boldsymbol{x}_{t} + \boldsymbol{\epsilon}_t$$
Here, $\boldsymbol{x} \in \mathbb{R}^{D}$ is the dynamical state variable, $A \in \mathbb{R}^{D\times D}$ are the linear dynamics and $\boldsymbol{\epsilon} \sim N(0,Q)$ is Gaussian noise, with $Q \in \mathbb{R}^{D\times D}$ the state noise covariance. We will first explore this equation and how the evolution of $\boldsymbol{x}$ depends on $A$ and $\boldsymbol{\epsilon}$ in more details.


1. Implement the deterministic part of the equation above, which takes a state $\boldsymbol{x}$ and dynamics matrix $A$, and generates the next state as $A \boldsymbol{x}$
2. Implement a function in Python that takes an initial condition $\boldsymbol{x}_0$ and the noise covariance $Q$ as input, and simulates $\boldsymbol{x}$ forward as a noisy dynamical system for T time-steps, according to the above equation.


In [ ]:
def simulate_trial(A, x0, T, Q):
    # x0 is the [D,] initial condition
    # T is the maximum number of timesteps we want to simulate
    # Q is the [D, D] noise covariance

    D = x0.shape[0]
    x = np.zeros([D, T])

    x[:, 0] = x0[:]
    for t in range(T-1):
        # x_t+1 = A * x_t + random multivariate normal noise with covariance Q
        # ############ Edit this #############################
        x[:, t + 1] =

    return x

In [ ]:
#@title Double click to see solution {display-mode: "form" }

def simulate_trial(A, x0, T, Q):
    # x0 is the [D,] initial condition
    # T is the maximum number of timesteps we want to simulate
    # Q is the [D, D] noise covariance

    D = x0.shape[0]
    x = np.zeros([D, T])

    x[:, 0] = x0[:]
    for t in range(T-1):
        # x_t+1 = A * x_t + random multivariate normal noise with covariance Q
        # ############ Edit this #############################
        x[:, t + 1] = A.dot(x[:, t]) + np.random.multivariate_normal(np.zeros([D]), Q, 1)

    return x

2.   Explore how different choices for $A$ affect the state evolution. Below there are two functions that implement a rotation matrix or a random symmetric matrix. Some things to explore:
*   Pick values for $\boldsymbol{x}_0$, $Q$ and $A$ and simulate the dynamical system using the functions you wrote before
*   What happens when $A$ is symmetric, or a rotation matrix? What looks different about the state evolution? How does that relate to the eigenvalues and eigenvectors of $A$?
*   What happens when eigenvalues of $A$ are real or complex?
*   How does the size of the largest eigenvalue affect the evolution?
*   What changes when $Q$ is large or small?

You should try to visualize the state evolution of $\boldsymbol{x}$ as a time series, and through a plot in 2D. Look at the functon `plot_flowfield`, which uses `streamplot` as well to see if you can visualize the flow-field with the paths of $\boldsymbol{x}_t$ overlaid on top.



In [ ]:
# implement a 2D rotation around an angle theta
def rotation(r, theta):
    return r*np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])

def symmetric(r):
    a = np.random.randn(2,2)
    u = np.linalg.svd(a)[0]
    e = r* np.array([1., 0.5]) # scale the eigenvalues by r
    return (u*e).dot(u.T)      # A real symmetric matrix fulfills U*E*U.T

In [ ]:
# make a linear system with a 2D rotation
x0 = np.array([-1, 1]) # initial condition
A_rot = rotation(0.99, 0.5) # dynamics matrix
A_sym = symmetric(0.99) # dynamics matrix
Q = 0.00 * np.eye(2) # innovations noise
T = 100

xpath_Rotation  = simulate_trial(A_rot, x0, T, Q)
xpath_Symmetric = simulate_trial(A_sym, x0, T, Q)

In [ ]:
# visualize simulated traces

# plot state evolution under the rotation dynamics
plt.plot(xpath_Rotation.T)
plt.xlabel('time')
plt.title('state evolution with 2D rotation dynamics')
plt.show()

# plot state evolution under the symmetric dynamics
plt.plot(xpath_Symmetric.T)
plt.xlabel('time')
plt.title('state evolution with 2D symmetric dynamics')
plt.show()


In [ ]:
def plot_flowfield(A,ngrid=20,w=2.5): #100, 2.5
    xv, yv = np.meshgrid(np.linspace(-w,w,ngrid), np.linspace(-w,w,ngrid))
    xygrid = np.hstack((xv.reshape(-1, 1), yv.reshape(-1, 1)))
    xy_out = A.dot(xygrid.T).T - xygrid # dt = x_t+1 - x_t
    U = xy_out[:,0].reshape(ngrid,ngrid)
    V = xy_out[:,1].reshape(ngrid,ngrid)
    speed = np.sqrt(U**2 + V**2)

    #drawing stream plot
    #plt.streamplot(xv, yv, U, V)
    plt.quiver(xv, yv, U, V)
    plt.axis('square')

In [ ]:
# plot flow field with rotational dynamics and overlaid state evolution in 2D
plt.title('rotational dynamics')

plot_flowfield(A_rot, ngrid=20,w=2.5)
# add marker for first point
plt.plot(xpath_Rotation[0,0],xpath_Rotation[1,0],'g.',markersize=20)
# add marker for last point
plt.plot(xpath_Rotation[0,-1],xpath_Rotation[1,-1],'r.',markersize=20)

# plot path
plt.plot(xpath_Rotation[0,:],xpath_Rotation[1,:],'k')

plt.xlabel('x1')
plt.ylabel('x2')
plt.show()

# plot flow field with symmetric dynamics and overlaid state evolution in 2D
plt.title('symmetric dynamics')

plot_flowfield(A_sym, ngrid=20,w=2.5)

# add marker for first point
plt.plot(xpath_Symmetric[0,0],xpath_Symmetric[1,0],'g.',markersize=20)
# add marker for last point
plt.plot(xpath_Symmetric[0,-1],xpath_Symmetric[1,-1],'r.',markersize=20)
# plot path
plt.plot(xpath_Symmetric[0,:],xpath_Symmetric[1,:],'k')
plt.xlabel('x1')
plt.ylabel('x2')
plt.show()

## **Exercise 2:** Simulating a low dimensional linear dynamical system

Now that we have a better understanding of how linear dynamical systems work, we can connect this to a model for high-dimensional neural data. To do this, we will use the same set up as we did for models like Factor Analysis. Here, the low-dimensional dynamical system is embedded within a high-dimensional space as follows

$$ x_{t+1} = A x_{t} + \epsilon_t, \quad x \in \mathbb{R}^D$$

$$ y_{t} = C x_{t} + \eta_t, \quad y \in \mathbb{R}^N$$

$C$ is a $N \times D$ matrix, where $D \le N$, and $\eta \sim N(0,R)$ is noise. We'll assume that $R$, which is an $N \times N$ matrix, is diagonal.

1.   Generate a time series for $x$ and $y$ and visualize your results. To do this you should write a function that simulates from the LDS model.
2.   Perform PCA on this data. Look at the dimensionality by plotting the singular values of the data covariance matrix. Visualise what the leading PC projections of the data look like.

In [ ]:
# make a linear system with a 2D rotation
x0 = np.array([-1, 1]) # initial condition
A = rotation(0.99, 0.5) # dynamics matrix
D = 2
N = 10
C = np.random.randn(N,D)
Q = 0.1 * np.eye(D) # innovations noise
R = 0.5*np.eye(N)
T = 100


def simulate_LDS(A, x0, T, Q, C, R):
    xpath = simulate_trial(A, x0, T, Q)
    # y_t = C * x_t + random multivariate normal noise with covariance R
    # ######### Edit this #####################
    ypath =
    ##########################################
    return ypath, xpath



In [ ]:
#@title Double click to see solution {display-mode: "form" }

# make a linear system with a 2D rotation
x0 = np.array([-1, 1]) # initial condition
A = rotation(0.99, 0.5) # dynamics matrix
D = 2
N = 10
C = np.random.randn(N,D)
Q = 0.1 * np.eye(D) # innovations noise
R = 0.5*np.eye(N)
T = 100


def simulate_LDS(A, x0, T, Q, C, R):
    xpath = simulate_trial(A, x0, T, Q)
    # y_t = C * x_t + random multivariate normal noise with covariance R
    # ######### Edit this #####################
    ypath = C.dot(xpath) + np.random.multivariate_normal(np.zeros(N), R, T).T
    ##########################################
    return ypath, xpath



In [ ]:
ypath, xpath = simulate_LDS(A, x0, T, Q, C, R) # simultate from the LDS model

In [ ]:
# plot ypath high-d output
plt.plot(ypath.T)
plt.title('high-d output time series')
plt.xlabel('time')
plt.show()


# plot xpath low-d latents
plt.plot(xpath.T)
plt.title('low-d latent time series')
plt.xlabel('time')
plt.show()


In [ ]:
#We can compute eigenvalues using the SVD of the data covariance, YY.T (since Y is centered), and we can ignore the scaling (n-1) since we will compute eigenvalue ratios
PCs = np.linalg.svd(ypath.dot(ypath.T))[0]
svds = np.linalg.svd(ypath.dot(ypath.T))[1] #singular values = eigenvalues in this case

# look at dimensionality
# plot the cumulative fraction of variance captured by each PC
# (hint: the fraction of variance in each PC is proportional to the singular value associated with that dimension)
plt.plot(svds/sum(svds),'o-')
plt.xlabel('dimension')
plt.ylabel('fraction of variance explained')
plt.show()


nPCs=4 # number of PCs to plot
for i in range(nPCs):
    plt.plot(PCs[:,i].dot(ypath))
plt.title('data PC projections')
plt.xlabel('time')
plt.show()

## **Exercise 3:** Maximum likelihood learning via the EM algorithm

Now that we have noisy, high-dimensional data, we want to find a way to estimate the underlying dynamics and latent states from this data. In our example, we made the system up and we know the ground-truth, but in real world applications we need to trust that our algorithm will estimate the right parameters, given the assumptions we made about where the data came from. In this exercise, we will work to build an algorithm that allows us to input observed measurements ($y$) and extract estimates for $A$, $x$, etc. This algorithm is called the Expectation Maximization (EM) algorithm. We've already seen it for probabilistic PCA and Factor Analysis and now we will extend it to our LDS model. EM is the foundation for a bunch of different Machine Learning algorithms.

Try to step through the functions below to try to understand what they are doing. `run_ssm_kalman` is implementing the Kalman Smoothing algorithm we discussed in the lecture.
1. What part of the EM algorithm does this correspond to?
2. What is the goal of running this function?

The details here are very technical, so don't worry too much about what the function does on the inside. If you are interested though, there is extra reading and some more detailed notes on EM in the reading material in the Google drive folder [this is advanced stuff!].

In [ ]:
def run_ssm_kalman(X, y_init, Q_init, A, Q, C, R, mode='smooth'):
    """
    Calculates kalman-smoother estimates of SSM state posterior.
    :param X:       data, [d, t_max] numpy array
    :param y_init:  initial latent state, [k,] numpy array
    :param Q_init:  initial variance, [k, k] numpy array
    :param A:       latent dynamics matrix, [k, k] numpy array
    :param Q:       innovariations covariance matrix, [k, k] numpy array
    :param C:       output loading matrix, [d, k] numpy array
    :param R:       output noise matrix, [d, d] numpy array
    :param mode:    'forw' or 'filt' for forward filtering, 'smooth' for also backward filtering
    :return:
    y_hat:      posterior mean estimates, [k, t_max] numpy array
    V_hat:      posterior variances on y_t, [t_max, k, k] numpy array
    V_joint:    posterior covariances between y_{t+1}, y_t, [t_max, k, k] numpy array
    likelihood: conditional log-likelihoods log(p(x_t|x_{1:t-1})), [t_max,] numpy array
    """
    d, k = C.shape
    t_max = X.shape[1]

    # dimension checks
    assert np.all(X.shape == (d, t_max)), "Shape of X must be (%d, %d), %s provided" % (d, t_max, X.shape)
    assert np.all(y_init.shape == (k,)), "Shape of y_init must be (%d,), %s provided" % (k, y_init.shape)
    assert np.all(Q_init.shape == (k, k)), "Shape of Q_init must be (%d, %d), %s provided" % (k, k, Q_init.shape)
    assert np.all(A.shape == (k, k)), "Shape of A must be (%d, %d), %s provided" % (k, k, A.shape)
    assert np.all(Q.shape == (k, k)), "Shape of Q must be (%d, %d), %s provided" % (k, k, Q.shape)
    assert np.all(C.shape == (d, k)), "Shape of C must be (%d, %d), %s provided" % (d, k, C.shape)
    assert np.all(R.shape == (d, d)), "Shape of R must be (%d, %d), %s provided" % (d, k, R.shape)

    y_filt = np.zeros((k, t_max))  # filtering estimate: \hat(y)_t^t
    V_filt = np.zeros((t_max, k, k))  # filtering variance: \hat(V)_t^t
    y_hat = np.zeros((k, t_max))  # smoothing estimate: \hat(y)_t^T
    V_hat = np.zeros((t_max, k, k))  # smoothing variance: \hat(V)_t^T
    K = np.zeros((t_max, k, X.shape[0]))  # Kalman gain
    J = np.zeros((t_max, k, k))  # smoothing gain
    likelihood = np.zeros(t_max)  # conditional log-likelihood: p(x_t|x_{1:t-1})

    I_k = np.eye(k)

    # forward pass

    V_pred = Q_init
    y_pred = y_init

    for t in range(t_max):
        x_pred_err = X[:, t] - C.dot(y_pred)
        V_x_pred = C @V_pred@C.T + R
        V_x_pred_inv = np.linalg.inv(V_x_pred)
        likelihood[t] = -0.5 * (np.linalg.slogdet(2 * np.pi * (V_x_pred))[1] +
                                x_pred_err.T.dot(np.linalg.solve(V_x_pred,x_pred_err)))

        K[t] = np.linalg.solve(V_x_pred.T, C@V_pred.T).T


        y_filt[:, t] = y_pred + K[t].dot(x_pred_err)
        V_filt[t] = V_pred - K[t]@C@V_pred

        # symmetrise the variance to avoid numerical drift
        V_filt[t] = (V_filt[t] + V_filt[t].T) / 2.0

        y_pred = A.dot(y_filt[:, t])
        V_pred = A @V_filt[t]@A.T + Q

    # backward pass

    if mode == 'filt' or mode == 'forw':
        # skip if filtering/forward pass only
        y_hat = y_filt
        V_hat = V_filt
        V_joint = None
    else:
        V_joint = np.zeros_like(V_filt)
        y_hat[:, -1] = y_filt[:, -1]
        V_hat[-1] = V_filt[-1]

        for t in range(t_max - 2, -1, -1):
            J[t] = np.linalg.solve(A@V_filt[t]@A.T + Q, A@V_filt[t].T).T

            y_hat[:, t] = y_filt[:, t] + J[t].dot((y_hat[:, t + 1] - A.dot(y_filt[:, t])))
            V_hat[t] = V_filt[t] + J[t]@(V_hat[t + 1] - A@V_filt[t]@A.T - Q)@J[t].T

        V_joint[-2] = (I_k - K[-1]@C)@A@V_filt[-2]

        for t in range(t_max - 3, -1, -1):
            V_joint[t] = V_filt[t + 1]@J[t].T + J[t + 1]@(V_joint[t + 1] - A@V_filt[t + 1])@J[t].T

    return y_hat, V_hat, V_joint, likelihood

The function `run_EM` implements the Expectation Maximization algorithm.  

Step through each line of this code and make sure you conceptually understand what it is supposed to do.

Can you map the different lines to the outline of the EM algorithm we talked about in the lecture?


If you are interested in understanding why the parameter updates look the way they do, there are more notes in the Google drive folder [this is advanced stuff!].

In [ ]:
def run_EM(X, y0, Q0, A, Q, C, R, maxiter=100, tol=1e-4):
    # X is the d x T data matrix
    # y0 is the initial condition for the latent state
    # Q0 is the initial Covariance of the first time step
    # A,Q,C,R are starting values for our parameters
    # maxiter is the maximum number of iterations to perform
    # tol is the convergence tolerance

    D, K = C.shape
    T = X.shape[1]
    loglike = []

    for i in tqdm(range(maxiter)):

        Y, V, Vj, likelihood = run_ssm_kalman(X, y0, Q0, A, Q, C, R, mode='smooth')
        loglike.append(sum(likelihood))

        # make sure that likelihood increases and check convergence
        if i > 0:
            diff = loglike[i] - loglike[i-1]
            if diff < 0:
                warnings.warn("decrease in log-likelihood detected")
        else:
            diff = []

        if i > 0 and abs(diff) < tol:
            print('EM has converged')
            break

        # perform parameter updates
        yy = sum(V) + Y@Y.T
        xy = X@Y.T
        rr1 = X@X.T
        aa1 = sum(V) + Y[:,:-1]@Y[:,:-1].T
        aa2 = sum(Vj) + Y[:,1:]@Y[:,:-1].T
        qq1 = sum(V[1:]) + Y[:,1:]@Y[:,1:].T

        # C update
        C = np.linalg.solve(yy.T, xy.T).T

        # A update
        A = np.linalg.solve(aa1.T,aa2.T).T

        # R update
        R = np.diag(np.diag((rr1 - C@xy.T)/T)) # diagonal matrix is more stable for real data application

        # Q update
        Q = (qq1 - aa2@A.T)/(T-1)

        # symmetrize covariances for numerical stability
        Q = 0.5*(Q + Q.T)
        R = 0.5*(R + R.T)

        # update initial conditions
        y0 = Y[:,0]
        Q0 = V[0] - y0.dot(y0.T)

    return A, Q, C, R, y0, Q0, Y, V, Vj, loglike

## **Exercise 4:** Applying EM to simulated data

Now that we have our EM algorithm ready to go, we can apply it to our simulated example and try to get our initial parameters back.

Start with an initial guess for the model parameters and run EM starting at these values.


In [ ]:
# initialize parameter settings

# dimensionality of our simulation from ealier
# D = 2
# N = 10

x0 = np.zeros(2)
Q0 = np.eye(2)
Ainit = 0.5*np.eye(2)
# Cinit = np.random.randn(N,D) # random initialization
Cinit = C[:] # initialization at the true parameter value
Rinit = np.eye(N)
Qinit = np.eye(D)

# run EM
Ahat, Qhat, Chat, Rhat, x0hat, Q0hat, X, V, Vj, loglike = run_EM(ypath, x0, Q0, Ainit, Qinit, Cinit, Rinit, maxiter=1000)

A good check in implementing EM algorithms is to make sure our log-likelihood increases with every iteration:

In [ ]:
# plot the loglikelihood for EM. Does it increase at each iteration?
plt.plot(loglike)
plt.title('log-likelihood')
plt.show()

## **Exercise 5:** Diagnostics and assesing goodness of fit

Now that we've ran our algorithm, how can we tell if we learned the right thing? Since we made up the data until now, we know what our true system looks like. We can first check that the output of our algorithm aligns with the ground truth.


1.   Plot the inferred latent variables along with the true underlying 2D system and see if they match.

2.   Does our estimate $\hat{A}$ have the same entries as our true generative $A$? Why would you expect this, or why not? Do the eigenvalues and eigenvectors match? Think about why looking at eigenvalues and eigenvectors is a better measure for assessing whether the parameters match

3. Run EM again but with different initializations for $C$ (true vs. random, see comments in code). What changes?


In [ ]:
plt.subplot(2,1,1)
plt.plot(X.T)
plt.plot(xpath.T,'--')
plt.title('inferred posterior means $\pm$ 2 std. dev. and true latent paths')
plt.xlabel('times')

for i in range(2):
    plt.fill_between(range(T), X[i,:] + 2*np.sqrt(V[:,i,i]),
                X[i,:]-2*np.sqrt(np.sqrt(V[:,i,i])), alpha=0.2)

plt.show()

plt.subplot(2,1,2)
plt.plot(X.T, xpath.T)
plt.title('inferred posterior means vs true latent paths')
plt.show()

In [ ]:
plt.subplot(1,2,1)
plot_flowfield(A)
plt.title('true flow field')

plt.subplot(1,2,2)
# plot the estimated flow-field
plot_flowfield(Ahat)
plt.title('estimated flow field')

plt.show()

In [ ]:
print('true dynamics:')
print(A)

print('estimated dynamics:')
print(Ahat)

In [ ]:
print('true dynamics eigen decomp:')
print(np.linalg.eig(A))

print('estimated dynamics eigen decom:')
print(np.linalg.eig(Ahat))

In real-world applications, we don't have access to the true underlying parameters. We want to look at data to estimate them and learn something new! So how can we assess whether our model is a good model? This is a tough question and still an active topic of debate and research.


3.   Think about what makes a model a "good" model. Can you come up with a list of features you'd like a good model to have?

A common measure is to assess a model by asking: how well does it fit the data? Another way to ask this question is: How much variability in our observed data can be explained by our model? This is sometimes also called "the fraction of variance explained".

4.   Below we have code to compute the fraction of variance that is explained by our model in the simulated data. Evaluate this for our current model fit and make a note of the value, then repeat the data generation and fitting procedure while increasing the level of noise in the latent states and in the data. What do you see now?

5.   Can you think of a way to report variance explained that would be less sensitive to noise variance?

6. What happens to the fraction of variance explained when you increase the number of latent states in the model? (but you don't change the latent state dimensionality of the simulation) Why is that?


In [ ]:
def varianceExplained(Y, Yhat):
    Yc = Y - np.mean(Y,axis=1)[:,None]
    diff = Y - Yhat
    RSS = np.trace(diff.dot(diff.T))
    TSS = np.trace(Yc.dot(Yc.T))
    Rsq = 1 - RSS/TSS
    return Rsq

In [ ]:
# compute the variance explained between the inferred poserior mean <y>_q = C <x>_q and the data
# <.>_q denotes the expected value under the posterior distribution
ypost = Chat.dot(X) # inferred poserior mean <y>_q = C <x>_q
varianceExplained(ypath, ypost)

How does this number change when we use a function to predict forward in time using the learned model rather than the inferred latent variables? How does that depend on the noise we used in simulating the system?

In [ ]:
def predict_path(A, x0, T):
    # x_t+1 = f
    # x0 is the [k,] initial condition
    # T is the maximum number of timesteps we want to simulate

    K = x0.shape[0]
    x = np.zeros([K, T])
    x[:, 0] = x0[:]
    for i in range(T-1):
        x[:, i + 1] = A.dot(x[:, i])

    return x

xpred = predict_path(Ahat, x0hat, T)

plt.plot(X.T)
for i in range(2):
    plt.fill_between(range(T), X[i,:] + 2*np.sqrt(V[:,i,i]),
                X[i,:]-2*np.sqrt(np.sqrt(V[:,i,i])), alpha=0.2)

plt.plot(xpred.T,'--')
plt.title('inferred posterior means and predictions forward from initial condition')
plt.show()


In [ ]:
yexpl_pred = Chat.dot(xpred)
varianceExplained(ypath,yexpl_pred)

## **Exercise 6:** Application to neural data

We now have a full algorithm to take high-dimensional data and infer a low-dimensional time series of latent states, and the dynamical system that explains their evolution over time from data. So far we have only worked with toy data, but we are now ready to put our algorithm to the test and see if it can extract interesting structure from neural data.

We will fit a dynamical system to trial-averaged data from motor cortex during movement execution. We've described the task in more detail in the lecture, so let's dig into the data now.

You will need to download the dataset from the google drive folder and upload it into this notebook using the code below.

In [ ]:
from google.colab import files

uploaded = files.upload()


This is a matlab file, so we need to read it into python and extract the relevant variables. The code below will do this:

In [ ]:
mat = scipy.io.loadmat('neuralData_ReachingTask.mat')

MO = int(mat['MO']) # movement onset
GC = int(mat['GC']) # go cue
Xneur = mat['X'].astype('float64') # C x T x N data matrix for C = 4 reach targets


We'll start by selecting data recorded during movements to a single reach target.

In [ ]:
# select a single target direction to fit
ctarg = 0
tstart = MO
ytarg = (Xneur[ctarg,GC:,:].T - np.mean(Xneur[ctarg,tstart:,:].T,axis=1)[:, None])/np.std(Xneur[ctarg,tstart:,:])
# selects data from tstart onwards

Now that we have loaded the data, we can get a sense of it's dimensionality and the kind of acitivity patterns motor cortex expresses during reaching by applying our old friend PCA to the data.

1. Compute the data covariance matrix and plot its singular values.
2. Plot the projections of the neural data along the leading principal components.

In [ ]:
PCs = np.linalg.svd(ytarg.dot(ytarg.T))[0]
svds = np.linalg.svd(ytarg.dot(ytarg.T))[1]

# look at dimensionality
plt.plot(svds[:20]/sum(svds),'o-')
plt.plot(np.cumsum(svds[:20]/sum(svds)),'o-')
plt.plot(0.9*np.ones(20,),'k')

plt.xlabel('dimension')
plt.ylabel('fraction of variance explained')
plt.show()

# plot PC projections
nPCs=4 # number of PCs to plot
for i in range(nPCs):
    plt.plot(PCs[:,i].dot(ytarg))
plt.show()

We also have our EM implementation now, so we can do more than using PCA!

Let's try to fit a dynamical system to our motor cortex data.
1. Choose a latent dimensionality K. What is a good value for this?
2. Initialize model paramters with random values
3. Run the EM algorithm.

In [ ]:
k = 10
d = ytarg.shape[0]

x0 = np.random.randn(k)
Q0 = np.eye(k)

# initialize parameters
Cinit = PCs[:,:k] # initialize with PCA
Qinit = 0.1*np.eye(k)
Rinit = 0.5*np.eye(d)
Ainit = 0.9*np.eye(k)

In [ ]:
# run EM
Ahat, Qhat, Chat, Rhat, x0hat, Q0hat, X, V, Vj, loglike = run_EM(ytarg, x0, Q0, Ainit, Qinit, Cinit, Rinit, maxiter=50)


4. Visualize the inferred posterior means. Do they look similar to the PC projections from before?


In [ ]:
plt.plot(X.T)
plt.title('inferred posterior means $\pm$ 2 std. dev')
# plot the inferred posterior means

# plot +/- 2 posterior std. around the means
for i in range(k):
    plt.fill_between(range(ytarg.shape[1]), X[i,:] + 2*np.sqrt(V[:,i,i]),
                X[i,:]-2*np.sqrt(V[:,i,i]), alpha=0.2)

plt.show()

5. Inspect the eigenvalues of the learned dynamics matrix. What do you notice about them? Is this what you expected based on our earlier exploration of properties of linear dynamics?

In [ ]:
evals = np.linalg.eig(Ahat)[0]
plt.subplot(1,2,1)
plt.plot(np.real(evals),np.imag(evals),'.')

By now, we have a pretty good understanding of linear dynamics and we managed to implement a whole machine learning algorithm and applied it to neural data. But we also only looked at a single target direction.

1. Look at the neural data for all targets using PCA. Do you think a single linear system could model this? Why? Why not?
2. Try to run the fitted model forward from an initial condition for a single or multiple targets. Do the model predictions look good?

In [ ]:
PCs = np.linalg.svd(Xneur.reshape(-1,d).T.dot(Xneur.reshape(-1,d)))[0]
svds = np.linalg.svd(Xneur.reshape(-1,d).T.dot(Xneur.reshape(-1,d)))[1]

# look at dimensionality
plt.plot(svds[:20]/sum(svds),'o-')
plt.plot(np.cumsum(svds[:20]/sum(svds)),'o-')
plt.plot(0.9*np.ones(20,),'k')

plt.xlabel('dimension')
plt.ylabel('fraction of variance explained')
plt.show()

# plot PC projections
nPCs=4 # number of PCs to plot
nTargets=4 # number of target directions
for i in range(nPCs):
  for c in range(nTargets):
      plt.plot(PCs[:,i].dot(Xneur[c].T))
  plt.title('PC %i'%i)
  plt.show()

In [ ]:
xpred = predict_path(Ahat, x0hat, ytarg.shape[1])
# function to predict forward from initial condition (you already wrote this above)

plt.plot(X.T)
for i in range(k):
    plt.fill_between(range(ytarg.shape[1]), X[i,:] + 2*np.sqrt(V[:,i,i]),
                X[i,:]-2*np.sqrt(V[:,i,i]), alpha=0.2)

plt.plot(xpred.T,'--')
plt.title('inferred posterior means and predictions forward from initial condition')
plt.show()

## **BONUS EXERCISE: Adding an offset to our model**

A limitation of linear systems is that they only have a single fixed point. This can be pretty restrictive, especially when movements to different targets produce pretty different-looking neural activity that might move to different points in the high-dimensional neural space.
There also different epochs of the task (e.g. motor planning and motor execution) and it would be surprising if a single dynamical system without inputs could model this well. How would the system know when to transition from rest to movement?

A simple but powerful extension to this simple linear model is to add an offset parameter that is condition specific and potentially time-varying:

$$ x^c_{t+1} = A x^c_{t} + b^c_t + \epsilon_t $$

1. What does this kind of extension change about our model? What flexibility do we now have that we didn't have before? Do you think this will alleviate some of the problems we saw in our predictions forward from an initial condition?

Let's try to get EM running for this model. We'll need to modify the E-Step to incorporate the extra paramter $b^c_t$, and derive a new update for $b^c$ for the M-step.

It turns out that a piecewise constant $b^c_t$ is a good choice for our motor cortex data. This means that both the preparatory period (the monkey sees a target and plans his movement to it but doesn't move yet) and the movement period (the monkey executes his arm reach) can be described by thge same linear dynamics, but the fixed point of the linear system moves around across task epochs (preparator vs. movement) and conditions (target direction).
